In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
# read data
df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
df_users = pd.read_csv("file\\ml-1m\\users.dat", sep="::", header=None, names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])

<ipython-input-3-66699cb8c506>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_ratings = pd.read_csv("file\\ml-1m\\ratings.dat", sep="::", header=None, names=["UserID", "MovieID", "Rating", "Timestamp"])
<ipython-input-3-66699cb8c506>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_movies = pd.read_csv("file\\ml-1m\\movies.dat", sep="::", header=None, names=["MovieID", "Title", "Genres"])
<ipython-input-3-66699cb8c506>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as reg

In [4]:
# merge data
df_ratings_movies = pd.merge(df_ratings, df_movies, on='MovieID')
df = pd.merge(df_ratings_movies, df_users, on="UserID")
df.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [5]:
# Delete unnecessary columns
data = df.copy()
data = data.drop(['Timestamp','Gender','Age','Occupation','Zip-code'], axis=1)
data.head()

,UserID,MovieID,Rating,Title,Genres
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


# Data preprocessing (extracting data only from users who have watched more than 50 movies)

In [6]:
count = data['UserID'].value_counts()
count_index = count[count > 50]
data_pre = data[data['UserID'].isin(count_index)]
data_pre

,UserID,MovieID,Rating,Title,Genres
3025,53,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
3026,53,661,5,James and the Giant Peach (1996),Animation|Children's|Musical
3027,53,914,5,My Fair Lady (1964),Musical|Romance
3028,53,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
3029,53,594,5,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
...,...,...,...,...,...
999762,158,922,3,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Film-Noir
999763,158,239,4,"Goofy Movie, A (1995)",Animation|Children's|Comedy|Romance
999764,158,376,5,"River Wild, The (1994)",Action|Thriller
999765,158,1812,5,Wide Awake (1998),Children's|Comedy|Drama


# Split train, test set

In [7]:
x = data_pre
x_train, x_test = train_test_split(x, test_size=0.05)

# Find popularity rankings by genre from training set

In [8]:
def genre_pop(dataset, genre, n):
    dataset_genres = dataset['Genres'].str.get_dummies("|")
    select_genre = dataset_genres[dataset_genres[genre]==1]
    genre_popmovie = dataset.loc[select_genre.index]
    genre_popmovie = genre_popmovie.reset_index()
    genre_popmovie_rank = genre_popmovie['MovieID'].value_counts()[:n]
    recomm_movie = genre_popmovie.loc[genre_popmovie_rank.index]
    return recomm_movie

genre_pop(x_train, 'Comedy', 10)

,index,UserID,MovieID,Rating,Title,Genres
2858,16802,264,2468,3,Jumpin' Jack Flash (1986),Action|Comedy|Romance|Thriller
1580,457662,78,205,1,Unstrung Heroes (1995),Comedy|Drama
2396,23175,331,34,5,Babe (1995),Children's|Comedy|Drama
1270,6986,136,141,3,"Birdcage, The (1996)",Comedy
1197,541307,346,3175,5,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi
1265,34154,504,2262,3,About Last Night... (1986),Comedy|Drama|Romance
2997,605624,386,441,4,Dazed and Confused (1993),Comedy
356,29574,425,1307,4,When Harry Met Sally... (1989),Comedy|Romance
1,50392,710,2245,3,Working Girl (1988),Comedy|Drama
2791,801987,894,785,5,Kingpin (1996),Comedy


# Find a user's genre ranking in training set

In [9]:
def user_genre(dataset, userid):
    user_data = dataset[dataset['UserID']==userid]
    user_data_genres = user_data['Genres'].str.get_dummies("|")
    user_genre_ranking = user_data_genres.sum().sort_values(ascending=False)
    return user_genre_ranking

user_genre(x_train, 54)

Comedy        35
Drama         11
Romance        4
Western        3
Children's     3
Sci-Fi         2
Crime          2
Action         2
War            1
Thriller       1
Mystery        1
Musical        1
Horror         1
Fantasy        1
Animation      1
Adventure      1
dtype: int64

# Recommend by genre

In [10]:
def user_genre_recommend(dataset, userid, n):
    genre_pref = user_genre(dataset, userid)
    recomm = genre_pop(dataset, genre_pref.index[0], n)
    return recomm

user_genre_recommend(x_train, 54, 5)

,index,UserID,MovieID,Rating,Title,Genres
2858,16802,264,2468,3,Jumpin' Jack Flash (1986),Action|Comedy|Romance|Thriller
1580,457662,78,205,1,Unstrung Heroes (1995),Comedy|Drama
2396,23175,331,34,5,Babe (1995),Children's|Comedy|Drama
1270,6986,136,141,3,"Birdcage, The (1996)",Comedy
1197,541307,346,3175,5,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi


# Performance Measurements

## Ignoring a genre

In [14]:
def popular_movie(dataset, n):
    movie_rank = dataset['MovieID'].value_counts()[:n]
    popular_movies = dataset.iloc[movie_rank.index]
    return popular_movies

def hitrate1(K):
    raw_ranking = popular_movie(data, K)
    pred_ranking = popular_movie(x_test, K)
    return raw_ranking['MovieID'].isin(pred_ranking['MovieID']).value_counts(normalize=True)[True]

s = 0
for i in range (100):
    s += hitrate1(100)
s /= 1000
s

0.011999999999999981

## Considering genre

In [15]:
def user_genre_recommend(dataset, userid, n):
    genre_pref = user_genre(dataset, userid)
    recomm = genre_pop(dataset, genre_pref.index[0], n)
    return recomm

def hitrate2(K):
    user = x_train.sample(n=1)['UserID'].values[0]
    raw_recomm = user_genre_recommend(data_pre, user, K)
    pred_recomm = user_genre_recommend(x_train, user, K)
    return raw_recomm['MovieID'].isin(pred_recomm['MovieID']).value_counts(normalize=True)[True]

s = 0
for i in range (100):
    s += hitrate2(100)
s /= 1000
s

0.024699999999999993